# Geographic analysis of public transportation and identifying poorly-covered areas

## Reading and handling population data

I start this task by loading and looking at the population data. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd

from visualization import visualize_pop_stops,  visualize_clusters, visualize_pop_reach
from utils import cluster_population, create_reach_area
import os

In [3]:
population_file = "bergen_pop.csv"
Bergen_pop_data = pd.read_csv(population_file)

In [4]:
Bergen_pop_data.describe()

,lat,lon
count,62245.000000,62245.000000
mean,57.665381,8.086164
std,11.675506,11.939029
min,4.642126,4.665742
25%,60.041785,5.157376
50%,60.332330,5.316041
75%,60.469010,5.519587
max,60.914139,61.229829


Here I observe that there although there is no analysis done on the column "Pop", which makes me believe some cleaning and handling is in order. I also wanted to check the total population to have a sanity check of the data, with the public information such as the total population of Bergen municipality. But with *sum(Bergen_pop_data["Pop"])*, I got the *TypeError: unsupported operand type(s) for +: 'int' and 'str'*, which means the data types for the population data is a mix of int and string, and we need to fix the data formats. 

For this purpose, I first tried  Bergen_pop_data["Pop"] = Bergen_pop_data["Pop"].astype(float), but it seems that in some cases, "," is used as decimal point, so I need to fix that first.

In [5]:
Bergen_pop_data["Pop"] = Bergen_pop_data["Pop"].str.replace(",", ".")

Bergen_pop_data["Pop"] = Bergen_pop_data["Pop"].astype(float)
sum(Bergen_pop_data["Pop"])

560962.6999999688

In [6]:
Bergen_pop_data.describe()

,Pop,lat,lon
count,62245.000000,62245.000000,62245.000000
mean,9.012173,57.665381,8.086164
std,18.723818,11.675506,11.939029
min,0.000000,4.642126,4.665742
25%,0.500000,60.041785,5.157376
50%,2.300000,60.332330,5.316041
75%,9.200000,60.469010,5.519587
max,387.300000,60.914139,61.229829


This seems to have solved the problem with formatting, but now the results are not consistent with common knowledge. The total population of Bergen municipality is recorded 289330 on 1.1.2023 (Source: [Bergen kommune website](https://www.bergen.kommune.no/omkommunen/fakta-om-bergen/befolkning/folketall-per-1-januar-2023)), but the result of summation over population is 560962.6999999688. Now that the format is correct, we try to see if there are repetetive values, which causes this.

In [7]:
Bergen_pop_data[Bergen_pop_data.duplicated(subset=["lat", "lon"], keep=False)]

,Pop,lat,lon


It seems that there are no duplicate data points and for each pair of (lat, lon), only one value exists in the data set. Now, let's try visualizing this data and see if we can then figure out what is going on. I found the folium module that can be used to visualize this data.

## Visualization of population data

In [8]:
vis_path = "./visualization_files/"
if not os.path.exists(vis_path):
    os.mkdir(vis_path)

visualize_pop_stops(
    pop_data=Bergen_pop_data,
    save_filename=f"{vis_path}1_population_map.html",
    circle_size_scale=50,
    zoom=2,
)

When visualizing the data, I see that there are some data points that are not located in Bergen, or indeed Norway, but they seem to be located in the Arabian sea (also can be seen in *Population_map_raw.png*). This is obviously an error when considering Bergen population. Looking back at the results of data description, I see that the values of quartiles for lat and lon have huge jumps, meaning there are outliers in these values. Also, the minimum and maximum values for lat and lon are almost the same, which cannot be the case in Norway. So, I believe there are some irrelevant data points here, and I limit the values of lat and lon to some values that I checked manually on google maps. I also remove the points in which the population is recorded zero, to prevent non-informative data points from appearing in visualization.

In [9]:
Bergen_pop_data = Bergen_pop_data[
    (Bergen_pop_data["lat"] > 57)
    & (Bergen_pop_data["lon"] < 10)
    & (Bergen_pop_data["Pop"] > 0)
]

In [10]:
Bergen_pop_data.describe()

,Pop,lat,lon
count,55440.000000,55440.000000,55440.000000
mean,9.792469,60.291194,5.406069
std,19.288433,0.300043,0.431403
min,0.100000,59.615511,4.665742
25%,0.700000,60.147616,5.154577
50%,2.800000,60.344938,5.304548
75%,10.300000,60.473218,5.474634
max,387.300000,60.914139,7.419300


Now the range of values in latitute and longitude seem reasonable and with far smaller value for standard deviation. I check again on the map to see if the problem is fixed.

In [11]:
visualize_pop_stops(
    pop_data=Bergen_pop_data,
    save_filename=f"{vis_path}2_population_map_Vestland.html",
    circle_size_scale=50,
    zoom=7,
)

Now the irrelevant data is removed, and I see only data points located in Vestland, in and around Bergen. I was still curious as to why the population count is much higher than that of Bergen kommune, but looking at the visualized map, it turns out that the data is given for much larger area than Bergen. It indeed covers large areas in Vestland. From what I see and using a map of districts in Vestland, the data includes the following districts: Nordhordland, Midhordland, Sunnhordland, Hardanger, and Voss, with the rough populations of $46000, 387000, 60000, 23000$, and $16000$, respectively. These numbers are from Wikipedia and mostly outdated, but in total they give the value $532000$, which is close enough to the summation of values in population after filtering out the irrelevant data, around $540212$, and means we can trust the quality of this data set now.

## Exploring the  Public Transport Stops data

In [12]:
stops_df = pd.read_csv("Bergen.stops/stops.txt")
stops_df.head()

,stop_id,stop_name,stop_lat,stop_lon,stop_desc,location_type,parent_station,wheelchair_boarding,vehicle_type,platform_code
0,NSR:Quay:100022,Hope nord,60.790205,5.034928,NaN,NaN,NSR:StopPlace:6647,NaN,700,NaN
1,NSR:Quay:100023,Litlås sør,60.793368,5.026605,NaN,NaN,NSR:StopPlace:6649,NaN,700,NaN
2,NSR:Quay:100024,Hopsvågen,60.780293,5.041870,NaN,NaN,NSR:StopPlace:12520,NaN,700,NaN
3,NSR:Quay:100025,Sverresplassen,60.629467,6.424300,NaN,NaN,NSR:StopPlace:13222,NaN,700,NaN
4,NSR:Quay:100026,Sverresplassen,60.629552,6.424155,NaN,NaN,NSR:StopPlace:13222,NaN,700,NaN


It seems that not all the columns of this data are relevant to the analysis here. I keep them for now and check for duplicate values when considering name, latitude, and longitude.

In [13]:
stops_df[
    stops_df.duplicated(subset=["stop_name", "stop_lat", "stop_lon"], keep=False)
].sort_values(by="stop_name")

,stop_id,stop_name,stop_lat,stop_lon,stop_desc,location_type,parent_station,wheelchair_boarding,vehicle_type,platform_code
10379,NSR:StopPlace:29686,Aga snuplass,59.845585,5.249156,NaN,1.0,NaN,NaN,700,NaN
1876,NSR:Quay:51041,Aga snuplass,59.845585,5.249156,NaN,NaN,NSR:StopPlace:29686,NaN,700,NaN
11109,NSR:StopPlace:30464,Aker CCB,60.409200,5.006249,NaN,1.0,NaN,NaN,700,NaN
3174,NSR:Quay:52405,Aker CCB,60.409200,5.006249,NaN,NaN,NSR:StopPlace:30464,NaN,700,NaN
2757,NSR:Quay:51960,Algrøyna snuplass,60.359554,4.952502,NaN,NaN,NSR:StopPlace:30213,NaN,700,NaN
...,...,...,...,...,...,...,...,...,...,...
1559,NSR:Quay:50708,Øyero,59.992840,5.539733,NaN,NaN,NSR:StopPlace:29502,NaN,700,NaN
14382,NSR:StopPlace:33853,Øygarden ungdomsskule,60.516533,4.909741,NaN,1.0,NaN,NaN,700,NaN
9180,NSR:Quay:58590,Øygarden ungdomsskule,60.516533,4.909741,NaN,NaN,NSR:StopPlace:33853,NaN,700,NaN
14026,NSR:StopPlace:33490,Øystese skule,60.388240,6.198062,NaN,1.0,NaN,NaN,700,NaN


Here I notice that the combination of *stop_name, stop_lat, stop_lon* has duplicates in some cases, and each seem to appear twice. The *stop_id* and *location_type* are different for duplicate rows, and it seems that the row with ID stating with *NSR:StopPlace* is the *parent_station* for the one whose ID starts with *NSR:Quay*. I am not exatly sure how these two are related, but for this task, I believe as long as they are in the same location, we can discard duplicates and keep only one. Also, given the tasks, we only need to keep the name, latitude, and longitude of each stop. Thus, I will drop the other features.

In [14]:
stops_df = stops_df.drop_duplicates(
    subset=["stop_name", "stop_lat", "stop_lon"], keep="first"
)
stops_df = stops_df[["stop_name", "stop_lat", "stop_lon"]]

Checking for missing values in each column:

In [15]:
stops_df.isnull().sum()

stop_name    0
stop_lat     0
stop_lon     0
dtype: int64

It appears to exist no missing values in the chosen features. Checking a general description of the data:

In [16]:
stops_df.describe()

,stop_lat,stop_lon
count,14084.000000,14084.000000
mean,60.321019,5.559716
std,0.323683,0.502514
min,59.394633,4.714310
25%,60.155222,5.227094
50%,60.368126,5.387175
75%,60.566246,5.778137
max,61.174890,8.205862


Now one of the two duplicate rows have been removed for each combination of *'stop_name','stop_lat', 'stop_lon'*, and the latitude and longitude of the stops are also in reasonable ranges.

## Visualization of Stops on map

In [17]:
visualize_pop_stops(
    save_filename=f"{vis_path}3_public_transport_stops_map.html",
    zoom=7,
    stops_data=stops_df,
)

I see that there are stops outside the scope of population points that are given, and outside the whole Vestland region, but I don't see a big problem with keeping them for now.

## Overlap of Stops and Population

The most straightforward thing would be to visualize stops and population in one figure. I use the same function, with both stops data and population data given.

In [18]:
visualize_pop_stops(
    save_filename=f"{vis_path}4_population_stops.html",
    zoom=7,
    pop_data=Bergen_pop_data,
    circle_size_scale=50,
    stops_data=stops_df,
)

I see that the overlap between the population and stops seems acceptable in most central areas. However, there is population in the south which doesn't seem to be covered by public transportation. This may be due to lack of stops data in these points, because some of the population points are located outside the Vestland region, or due to lack of reachability in these areas. I have the same observation in the western area on the islands. We can investigate this further when the reach areas are determined and not_covered population is more clearly defined.

## Identifying the reach area

For this task, I found the *shapely* library to define the reach area for each stop point and finally merge them into one polygon. The library has a method for each *point*, called *buffer*, but the measure is in degrees, because we are working with latitude and longitude. So, I will estimate the value in degrees equivalent to 500 meters.

In [19]:
eq_500meters = 500/111320

In [20]:

stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df["stop_lon"], stops_df["stop_lat"]),
)
stops_gdf.set_crs(epsg=4326)

reach_area_gdf = create_reach_area(
    stops=stops_gdf, buffer_degrees=eq_500meters
)

reach_area_gdf = reach_area_gdf.set_crs(epsg=4326)

In [21]:
# reach_area_gdf.explore()

The areas shown here look reasonable, so we continue the analysis with the *reach_area_gdf* from the above.

## Visualization of reach areas with population

We want to see the overlap of reach areas and population points. A function is defined for this purpose.

In [22]:
visualize_pop_reach(
    save_filename=f"{vis_path}5_reach_population.html",
    reach_area_gdf=reach_area_gdf,
    pop_data=Bergen_pop_data,
    circle_size_scale=50,
)

## Identifying the areas not covered by public transportation stops

I first convert the population data to a Geopandas Dataframe, in order to be able to use the capabilities of the *sjoin* method. Two versions of this function are used: the first one enables us to find not covered population using the option *within*, and the second allows us to have the covered population in a separate Dataframe.  

In [23]:
population_gdf = gpd.GeoDataFrame(
    Bergen_pop_data,
    geometry=gpd.points_from_xy(Bergen_pop_data["lon"], Bergen_pop_data["lat"]),
)
population_gdf = population_gdf.set_crs(epsg=4326)

covered_population = gpd.sjoin(population_gdf, reach_area_gdf, how="left", predicate="within")
only_covered_population = gpd.sjoin(population_gdf, reach_area_gdf, how="inner")
not_covered_population = population_gdf[covered_population["index_right"].isna()]

I use the same function with both dataframes: the covered population and the not_covered population, to make sure that the covered population points are indeed located within the reach area and the not covered population lie outside the reach area. 

In [24]:
visualize_pop_reach(
    save_filename=f"{vis_path}6_reach_covered_population.html",
    reach_area_gdf=reach_area_gdf,
    pop_data=only_covered_population,
    circle_size_scale=50,
)
visualize_pop_reach(
    save_filename=f"{vis_path}7_reach_not_covered_population.html",
    reach_area_gdf=reach_area_gdf,
    pop_data=not_covered_population,
    circle_size_scale=50,
)

I see this is the case.

To analyze the covered and not-covered populations, we can first look at the total population that lives within 500 meters of the public transportation, and consider what percentage of the total recorded population lies in this category:

In [25]:
covered_pop = only_covered_population["Pop"].sum()
total_pop = Bergen_pop_data["Pop"].sum()
print(f"The total population covered by the public transport is {int(covered_pop)}.")
print(f"This accounts for {100*(covered_pop/total_pop):.2f}% of the population in the reported area.")

The total population covered by the public transport is 424215.
This accounts for 78.14% of the population in the reported area.


From the figures, I also observe that there is a good deal of coverage within the city center, but interestingly, there exists a dense population right in the middle of city center which remains not covered. It seems that most of these areas are on the mountains, specifically Fløyen. There is a funicular called Fløibanen that can be used here, but it is not part of public transportation system and thus not recorded here. Also, the trip frequency of Fløibanen can make it unrealistic as a permanent mode of transportation.

## Identifying clusters of not covered population

To determine the values for *eps* and *min_samples*, I did some trial and errors with the clustering function, and check how reasonable the clusters look on the map, the number of clusters and the proportion of data points recognized as noise. I believe under $20\%$ of unclassifies/noise data should be acceptable, if this population is scattered. I also use the population of each point as *sample_weight* in *fit_predict* method.

In [26]:
clusters = cluster_population(pop_data=not_covered_population, eps=0.0001, min_samples=10)

not_covered_population = not_covered_population.assign(cluster = clusters)

Removing the data points recognized as noise, and checking the proportion of clustered data, these values for the parameters seem fine.

In [27]:
clustered_not_covered_population = not_covered_population[not_covered_population["cluster"] > -1]
print(len(clustered_not_covered_population)/len(not_covered_population))
n_clusters = clusters.max()+1
n_clusters

0.8616342571326494


637

Let us see how it looks on the map:

In [28]:

visualize_clusters(
    save_filename=f"{vis_path}8_clusters.html",
    pop_data=clustered_not_covered_population,
    circle_size_scale=50,
    n_clusters = n_clusters
)

From this figure, it can be seen that most of the not covered clusters lie on the smaller towns outside Bergen or on the islands in the west of Bergen. I would like to focus more on the most populated clusters and see where they are located: 

In [29]:
most_populated = (
    clustered_not_covered_population.groupby("cluster")
    .agg({"Pop": "sum", "lat": "mean", "lon": "mean"})
    .sort_values(by="Pop", ascending=False)
)
largest_clusters = most_populated[:10].index

In [30]:
most_populated_not_covered = clustered_not_covered_population[
    clustered_not_covered_population["cluster"].isin(largest_clusters)
]

In [31]:
visualize_clusters(
    save_filename=f"{vis_path}9_ten_largest_clusters.html",
    pop_data=most_populated_not_covered,
    circle_size_scale=50,
)

In [32]:
top_ten_clusters = most_populated[:10]

From the data from the above, I identified the 10 most populated not covered areas in Bergen. They can be seen in the following dataframe and with the average latitude and longitude, can be easily identified using a map. We might be able to consider them as prioritized when there is a decision about increasing the reachability of public transportation. 

In [33]:
top_ten_clusters

,Pop,lat,lon
cluster,,,
70,9611.6,60.310080,5.334411
7,6948.8,60.312782,5.236010
36,5955.5,60.803673,5.039145
13,4879.3,59.650816,6.360584
11,4106.0,60.440484,5.134359
68,3709.4,59.764479,5.494526
34,2747.9,60.175118,5.448380
53,2537.2,60.407853,5.344926
6,1807.3,60.456943,5.205810


## Future work

There is a lot more that can be done here. For example, the reach area can be thought out more carefully, considering different values as distance and performing the analysis to compare. We can also consider the walking time to the stops, including different difficulty to go uphill and downhill. The clusters can also be considered with more care, especially when determining the *eps* parameter. 